# Getting healthy PBMC data from
[Single-cell RNA sequencing unveils the shared and the distinct cytotoxic hallmarks of human TCRVδ1 and TCRVδ2 γδ T lymphocytes](https://www.pnas.org/content/116/24/11906)

Used Chromium V2 chemistry for scRNA-seq of healthy blood PBMCs

* Donor1 - SRR8718858
* Donor1 - SRR8718859
* Donor2 - SRR8718860
* Donor2 - SRR8718861
* Donor3 - SRR8718862
* Donor3 - SRR8718863


In [ ]:
%%bash 

#Pull healthy PBMC scRNA-seq Chromium V2 SRA file
cd /input_dir/crispr_analysis/input_reads
prefetch -o /input_dir/crispr_analysis/input_reads/pbmc/SRR8718860.sra SRR8718860


In [ ]:
%%bash
#Dump SRA file into reads using parallel-fastq-dump (preferred)
parallel-fastq-dump -t 8 --tmpdir /input_dir/crispr_analysis/temp/ \
    -s /input_dir/crispr_analysis/input_reads/pbmc/SRR8718860.sra \
    --dumpbase --clip --readids --gzip \
    --read-filter pass --split-files --origfmt \
    --outdir /input_dir/crispr_analysis/input_reads/pbmc


In [ ]:
#Dump SRA file into reads using fasterq (meh atm, no gzip)
fasterq-dump --skip-technical -e 8 -s -3 -O /fast_dir/seq_data/input_reads/pbmc \
    -t /fast_dir/seq_data/raw_sra/temp \
    SRR8718860


In [ ]:
%%bash

cd /data_dir/crispr_analysis/input_reads/pbmc
#Filter reads and check quality, turn on UMI processing and max read length=0 to not toss first read
fastp -p 12 -i SRR8718860_pass_1.fastq.gz \
      -I SRR8718860_pass_2.fastq.gz \
      -o SRR8718860_filt_1.fq.gz \
      -O SRR8718860_filt_2.fq.gz \
      -h SRR8718860_fastp.html \
      -j SRR8718860_fastp.json 
#      --umi --umi_loc read1 --umi_len 26 -l 0   #Finds and appends Barcode to read name, not what i want


In [ ]:
%%bash
#Set directory
#mkdir /input_dir/crispr_analysis/alignment_out/pbmc/
cd /data_dir/crispr_analysis/input_reads/pbmc


In [ ]:
%%bash

#Quantify using k-mer based Salmon Alevin
salmon alevin -l ISR \
              -1 SRR8718860_filt_1.fq.gz \
              -2 SRR8718860_filt_2.fq.gz \
              --chromium \
              -i /data_dir/corona_analysis/annotations/human/salmon_ann/salmon_hg38_index \
              -p 10 --dumpMtx \
              --mrna /data_dir/corona_analysis/annotations/human/gencode_mt.txt \
              --rrna /data_dir/corona_analysis/annotations/human/rRNA_ensembl.txt \
              -o /data_dir/crispr_analysis/alignment_out/pbmc/donor_2_1 \
              --tgMap /data_dir/corona_analysis/annotations/human/salmon_grch38_gencode_tran2gene.txt


In [ ]:
%%bash

#Quantify using alignment based STAR
STAR --runMode alignReads \
     --genomeDir /input_dir/corona_analysis/annotations/human/STAR_ix \
     --outSAMtype BAM SortedByCoordinate --runThreadN 14 \
     --outFileNamePrefix /input_dir/crispr_analysis/alignment_out/pbmc/donor_1_1_star/ \
     --outTmpDir /input_dir/crispr_analysis/temp/star1 \
     --outReadsUnmapped Fastx --limitBAMsortRAM 90000000000 \
     --outWigType wiggle --quantMode TranscriptomeSAM --readFilesCommand zcat \
     --soloType CB_UMI_Simple --soloCBwhitelist /input_dir/corona_analysis/annotations/human/scRNA_10x_v2_whitelist.txt \
     --soloFeatures Gene --outSAMattributes NH HI nM AS CR UR CB UB GX GN sS sQ sM \
    --readFilesIn SRR8718860_filt_2.fq.gz SRR8718860_filt_1.fq.gz

